<a href="https://colab.research.google.com/github/vishnu9358862212/AIML-EXP/blob/main/Sentimental_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
import string
import re
import pandas as pd
import numpy as np

from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE

In [ ]:
# 📦 Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving YoutubeCommentsDataSet.csv to YoutubeCommentsDataSet (1).csv


In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded['YoutubeCommentsDataSet (1).csv']))

In [ ]:
# Drop missing comments
df.dropna(subset=["Comment"], inplace=True)

In [ ]:
# 🧹 Advanced Preprocessing
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [ ]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    return wordnet.NOUN


In [ ]:
def preprocess_text(text):
    text = str(text).lower().strip()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"\d+", "", text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)
    lemmatized = [lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in pos_tags if word not in stop_words]
    return ' '.join(lemmatized)


In [ ]:
# 🧼 Apply Preprocessing
# 📦 Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger_eng') # Download the correct resource


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [ ]:
df['Cleaned_Comment'] = df['Comment'].apply(preprocess_text)

In [ ]:
# 🔠 TF-IDF Vectorization
vectorizer = TfidfVectorizer(
    max_features=10000,
    ngram_range=(1, 2),
    sublinear_tf=True,
    min_df=3,
    max_df=0.9
)
X = vectorizer.fit_transform(df['Cleaned_Comment'])
y = np.array(df['Sentiment'])


In [ ]:
# 🔀 Split Dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# ⚖️ Balance Data with SMOTE
smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)


In [ ]:
# 🧠 Train Logistic Regression
model = LogisticRegression(max_iter=2000)
model.fit(X_train_bal, y_train_bal)


LogisticRegression(max_iter=2000)

In [ ]:
# 📊 Evaluate
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.7378164987748435

Classification Report:
               precision    recall  f1-score   support

    negative       0.45      0.62      0.52       441
     neutral       0.59      0.60      0.60       912
    positive       0.89      0.81      0.85      2320

    accuracy                           0.74      3673
   macro avg       0.64      0.68      0.66      3673
weighted avg       0.76      0.74      0.75      3673



In [ ]:
# 🔮 Sentiment Prediction Function
def predict_sentiment(text):
    cleaned = preprocess_text(text)
    vectorized = vectorizer.transform([cleaned])
    return model.predict(vectorized)[0]


In [ ]:
# 🧪 Example Prediction
sample = "I absolutely love this product!"
print("Predicted Sentiment:", predict_sentiment(sample))

Predicted Sentiment: positive


In [42]:
import pickle

# Save model
with open("model.pkl", "wb") as f:
    pickle.dump(model, f)

# Save vectorizer
with open("vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)


In [43]:
from google.colab import files

files.download("model.pkl")
files.download("vectorizer.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>